In [42]:
import os
import pandas as pd
from Levenshtein import distance as lev

path = 'C:/Users/Konstantin/Documents/Omdena project/data/daily_air_quality_data_2017-2021/'

def read_xslx_files(file_path, nrows = None, header=0):
    excel_data = pd.read_excel(file_path, nrows=nrows, header=header)
    df = pd.DataFrame(excel_data)
    return df

def get_files_names(dir, format='.xlsx'):
    file_list = []
    for root, dirs, files in os.walk(dir, topdown=False):
        for name in files:
            if name.endswith(format):
                file_name = (os.path.join(root, name))
                file_list.append(file_name)
    return file_list

df_stantion_code = read_xslx_files(path + 'Measuring_stations_codes.xlsx', None, 0)
codes = df_stantion_code['Old code'].tolist() + df_stantion_code['New code'].tolist()

for dir_name in ['NO2', 'O3', 'PM10', 'PM25']:
    file_names_list = get_files_names(path + dir_name)
    for file_name in file_names_list:
        df = read_xslx_files(file_name, 1)
        if df.columns[1] == 1 or df.columns[1] == '1':
            df = read_xslx_files(file_name, None, 1)
        else:
            df = read_xslx_files(file_name, None, 0)
        df['flag'] = df.iloc[:, 0].apply(lambda x: 1 if str(x)[:2] == '20' else 0)
        df = df[df['flag'] == 1].drop(columns='flag').rename(columns={'Unnamed: 0':'date', 'Code of the station':'date'})
        if file_name == file_names_list[0]:
            df_res = df
        else:
            df_res = pd.concat([df_res, df], axis=0, join='outer')
        df_res = df_res.reset_index().drop(columns='index')
        
    check_list = []
    for first_name in df_res.columns:
        for second_name in df_res.drop(columns=first_name):
            diff = lev(first_name, second_name)
            if diff <= 2 and (second_name, first_name) not in check_list \
                         and df_res[(df_res[first_name].notna()) & (df_res[second_name].notna())].shape[0] == 0:
                check_list.append((first_name, second_name))
        
    for elem in check_list:
        if elem[0] in codes:
            n = 0
        else:  
            n = 1
        try:
            df_res[elem[n]][df_res[elem[n]].isna()] = df_res[elem[abs(n - 1)]][df_res[elem[n]].isna()]
            df_res = df_res.drop(columns=elem[abs(n - 1)])
        except:
            pass
    avg_time = '_1h_avg' if dir_name == 'O3' else '_24h_avg'
    df_res.to_csv(f'{path + "cleaned/" + dir_name + avg_time + "_polution_2017-2021"}.csv', index=False)